# List of Exercises 1 - Exercise 9

*Student: Luigi Lucas de Carvalho Silva / luigi.lcsilva@gmail.com*

First of all, let us import some useful packages.

In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import os
import scipy.integrate as integrate
import scipy.special as special
import scipy.interpolate as interpolate
import scipy.stats as stats

**Defining the distributions.**

Common parameters.

In [2]:
### Common parameters for the uniform distribution.
initial_unif=0
final_unif=1

### Common parameters for the gaussian.
mu_gauss = 0
sigma_gauss = 1

### Common parameters for the third distribution.
x_0 = 10
sigma_third = 5

Uniform distribution.

In [3]:
### Definition of the uniform distribution.
def uniform_dist(x, i, f):
    'i and f are the beginning and the end of the interval.'
    if i <= x <= f:
        y = 1/(f-i)
    else:
        y = 0
    return y

Gaussian distribution.

In [4]:
### Gaussian distribution definition.
def gaussian(x, mu, sigma):
    g = (1/(np.sqrt(2*np.pi)*sigma))*np.exp(-(x-mu)**2/(2*sigma**2)) 
    return g

Gaussian CDF.

In [5]:
### "Exact" gaussian CDF function definition.
def gaussian_cdf(x, mu, sigma):
    cdf = (special.erf((x-mu)/(np.sqrt(2)*sigma)) + 1)/2 
    return cdf

Gaussian inverse CDF.

In [6]:
### Points for interpolation. Here I get x points that are inside n_sigma*sigma around the mean.
n_sigma_gauss = 8
step_gauss = 2*n_sigma_gauss*sigma_gauss/200000

x_min_gauss = mu_gauss-n_sigma_gauss*sigma_gauss
x_max_gauss = mu_gauss+n_sigma_gauss*sigma_gauss

x_values_gauss = np.arange(x_min_gauss, x_max_gauss, step_gauss)
cdf_values_gauss = gaussian_cdf(x_values_gauss, mu_gauss, sigma_gauss)

### Defining the inverse CDF from a cubic interpolation using scipy.interpolate:
gaussian_inv_cdf = interpolate.interp1d(cdf_values_gauss, x_values_gauss, kind='linear')

Third distribution of the first exercise.

In [7]:
### Definition of the third distribution of the first exercise.
def exerc1_distrib3(x, x_0, sigma):
    y = (1/(np.sqrt(2*np.pi)*sigma))*(np.exp(-(x-x_0)**2/(2*sigma**2)) - np.exp(-(x+x_0)**2/(2*sigma**2)))*(1/special.erf(x_0/(np.sqrt(2)*sigma)))
    return y

Third distribution CDF.

In [8]:
### Defining the third function CDF. For this, I integrate the function in a interval around some sigmas of the
### mean, and then I interpolate these values to generate the CDF.

### Here is the x interval where I will do the integration. I am integrating regions in steps of "step",
### because I will make a "cumsum()" in the integrals array for obtaining the values of the CDF.
sigma = sigma_third
n_sigma = 8                                    #Numbers of sigma for the interpolation to include.
interval_division = 200000            #Number of divisions of the total region.
step = 2*n_sigma*sigma/interval_division
### I take care here for not taking negative values in the integration limits.
if (x_0-n_sigma*sigma)>0:
    x_comp = np.arange(x_0-n_sigma*sigma, x_0+n_sigma*sigma+step, step)
else:
    x_comp = np.arange(0, x_0+n_sigma*sigma+step, step)
        
### Now, I do the integration in each region and apply the "cumsum()".
y_cdf = np.array([tup[0] for tup in [integrate.quad(exerc1_distrib3, a, b, args=(x_0, sigma))
                                     for a, b in [(a, b) for a, b in zip(x_comp, x_comp[1:len(x_comp)])]]]
                                     + [0]).cumsum()

### Finally, I obtain the interpolated function. I exclude the last point because it usually leads to
### problems (I think it has to do with the integration limits definition).
scipy_exerc1_distrib3_cdf = interpolate.interp1d(x_comp[0:-1], y_cdf[0:-1], kind='linear')

Third distribution inverse CDF.

In [9]:
### Points for interpolation. Here I get x points that are inside n_sigma*sigma around the mean.
n_sigma_third = 8
step_third = 2*n_sigma_third*sigma_third/200000

if x_0-n_sigma_third*sigma_third>0:
    x_min_third = x_0-n_sigma_third*sigma_third
    x_max_third = x_0+n_sigma_third*sigma_third
else:
    x_min_third = 0
    x_max_third = x_0+n_sigma_third*sigma_third

x_values_third = np.arange(x_min_third, x_max_third, step_third)
cdf_values_third = scipy_exerc1_distrib3_cdf(x_values_third)

### Defining the inverse CDF from a cubic interpolation using scipy.interpolate:
scipy_exerc1_distrib3_inv_cdf = interpolate.interp1d(cdf_values_third, x_values_third, kind='linear')

**Defining the estimators.**

In [10]:
### First estimator (add up all and divide by N):
def est_1(x):
    est_1 = sum(x)/len(x)
    return est_1

### Second estimator (add up the first 10 numbers and divide by 10):
def est_2(x):
    est_2 = sum(x[:10])/10
    return est_2

### Third estimator (add up all and divide by N-1):
def est_3(x):
    est_3 = sum(x)/(len(x) - 1)
    return est_3

### Fourth estimator (the answer is always 1.8):
def est_4(x):
    return 1.8

### Fifth estimator (multiply all and take the Nth root):
def est_5(x):
    if any(n < 0 for n in x) == True: # If a value is negative, we can not perform the geometric mean.
        est_5 = np.nan
    else:
        est_5 = stats.mstats.gmean(x)
    return est_5

### Sixth estimator (the answer is the mode):
def est_6(x):
    est_6 = stats.mode(x)
    return float(est_6[0])

### Seventh estimator (add the min and the max and divide by 2):
def est_7(x):
    est_7 = (max(x) + min(x))/2
    return est_7

### Eighth estimator (add the second, fourth, sixth, etc., and divide by N/2 for N even, (N-1)/2 for N odd):
def est_8(x):
    N = len(x)
    summation = sum(x[i] for i in range(1,len(x),2)) #The second element is x[1], the fourth is x[3]...
    if N%2 == 0:
        est_8 = summation/(N/2)
    else:
        est_8 = summation/((N-1)/2)
    return est_8

## Generating random numbers for each distribution and computing the mean.

In [11]:
rnd_quantity = 10000

### Uniform random numbers ###
np.random.seed(seed=1) # Seed
x_unif = np.random.random_sample(size=rnd_quantity)

### Gaussian random numbers ###
np.random.seed(seed=2) # Seed
unif_random_gauss = np.random.random_sample(size=rnd_quantity)

x_gauss = gaussian_inv_cdf(unif_random_gauss)

### Third distribution random numbers ###
np.random.seed(seed=3) # Seed
unif_random_third = np.random.random_sample(size=rnd_quantity)

x_third = scipy_exerc1_distrib3_inv_cdf(unif_random_third)

Computing the mean.

In [12]:
all_func = [est_1, est_2, est_3, est_4, est_5, est_6, est_7, est_8]
all_func_names = ['est_1', 'est_2', 'est_3', 'est_4', 'est_5', 'est_6', 'est_7', 'est_8']

means_unif = []
means_gauss = []
means_third = []

for fn in all_func:
    means_unif.append(fn(x_unif))
    means_gauss.append(fn(x_gauss))
    means_third.append(fn(x_third))

Saving in a dataframe for better visualization.

In [13]:
df_means = pd.DataFrame()

### Defining new dataframes with the data.
df_means_func = pd.DataFrame(data=all_func_names)
df_means_unif = pd.DataFrame(data=means_unif)
df_means_gauss = pd.DataFrame(data=means_gauss)
df_means_third = pd.DataFrame(data=means_third)

### Concatening these dataframes with the main one.
df_means = pd.concat((df_means,df_means_func),axis=1)
df_means = pd.concat((df_means,df_means_unif),axis=1)
df_means = pd.concat((df_means,df_means_gauss),axis=1)
df_means = pd.concat((df_means,df_means_third),axis=1)

df_means.columns = ['Estimator','Uniform','Gaussian','Third']

df_means

,Estimator,Uniform,Gaussian,Third
0,est_1,0.497996,-0.020156,10.433170
1,est_2,0.314629,-0.445286,9.925060
2,est_3,0.498046,-0.020158,10.434214
3,est_4,1.800000,1.800000,1.800000
4,est_5,0.366370,NaN,9.261889
5,est_6,0.000097,-3.802497,0.056664
6,est_7,0.499986,-0.014811,13.977645
7,est_8,0.499689,-0.009789,10.426581


## Checking Consistency

An estimator is consistent if it tends to the true value as the number of data values tends to infinity.

Let us generate random numbers for each distribution again, but this time I will generate a huge number.

In [14]:
###########################################
###### GENERATING THE RANDOM NUMBERS ######
###########################################
rnd_quantity = 1000000

### Uniform random numbers ###
np.random.seed(seed=1) # Seed
x_unif = np.random.random_sample(size=rnd_quantity)

### Gaussian random numbers ###
np.random.seed(seed=2) # Seed
unif_random_gauss = np.random.random_sample(size=rnd_quantity)

x_gauss = gaussian_inv_cdf(unif_random_gauss)

### Third distribution random numbers ###
np.random.seed(seed=3) # Seed
unif_random_third = np.random.random_sample(size=rnd_quantity)

x_third = scipy_exerc1_distrib3_inv_cdf(unif_random_third)

###########################################
############# COMPUTING MEANS #############
###########################################
means_unif = []
means_gauss = []
means_third = []

for fn in all_func:
    means_unif.append(fn(x_unif))
    means_gauss.append(fn(x_gauss))
    means_third.append(fn(x_third))

Now, let us compute the expected value of the third distribution.

In [15]:
def third_argument(x, x_0, sigma):
    argument = x*(1/(np.sqrt(2*np.pi)*sigma))*(np.exp(-(x-x_0)**2/(2*sigma**2)) - np.exp(-(x+x_0)**2/(2*sigma**2)))*(1/special.erf(x_0/(np.sqrt(2)*sigma)))
    return argument

expected_third_array = integrate.quad(third_argument, 0, 300, args=(x_0, sigma))

expected_third = expected_third_array[0]

Now, let us check consistency.

In [16]:
df_consistency = pd.DataFrame()

df_consistency['Estimator'] = pd.Series(data=all_func_names)
df_consistency['Unif. Estimated-Expected'] = pd.Series(data=((x - 0.5) for x in means_unif))
df_consistency['Gauss. Estimated-Expected'] = pd.Series(data=((x - mu_gauss) for x in means_gauss))
df_consistency['Third. Estimated-Expected'] = pd.Series(data=((x - expected_third) for x in means_third))

df_consistency

,Estimator,Unif. Estimated-Expected,Gauss. Estimated-Expected,Third. Estimated-Expected
0,est_1,-5.174587e-05,-0.001729,0.006569
1,est_2,-1.853707e-01,-0.445286,-0.551633
2,est_3,-5.124592e-05,-0.001729,0.006580
3,est_4,1.300000e+00,1.800000,-8.676692
4,est_5,-1.319559e-01,NaN,-1.201808
5,est_6,-4.999997e-01,-4.764294,-10.458588
6,est_7,-7.180147e-08,-0.092551,6.940189
7,est_8,8.290711e-05,-0.001069,0.003938


As we can see, estimators 1, 3 and 8 are consistent.

## Checking Bias

An estimator is unbiased if its expectation value is equal to the true value.

The Central Limit Theorem says, basically, that the distribution of some variable that is obtained from a sum of a lot of variables tends to a Gaussian distribution. 

https://people.richland.edu/james/lecture/m170/ch07-clt.html

So, we can estimate the expected value of some estimator, E(T), by taking the average value of T computed from all possible samples of a given size that may be drawn from the population. 

https://stats.stackexchange.com/questions/493239/how-to-calculate-the-expected-value-of-an-estimator

https://sites.warnercnr.colostate.edu/gwhite/wp-content/uploads/sites/73/2017/04/ExpectedValue.pdf

Let us do it. First, I will generate a lot of samples of some fixed size for each distribution and compute the estimators values in each sample.

In [17]:
samples_quantity = 100000

means_unif_all = []
means_gauss_all = []
means_third_all = []

j=0
for i in range(0, samples_quantity):  
    ### Dicts for saving the means
    means_unif = []
    means_gauss = []
    means_third = []
    
    ###########################################
    ###### GENERATING THE RANDOM NUMBERS ######
    ###########################################
    ### Quantity of numbers by sample
    rnd_quantity = 30
    
    ### Uniform random numbers ###
    np.random.seed(seed=j)
    x_unif = np.random.random_sample(size=rnd_quantity)
    
    ### Gaussian random numbers ###
    np.random.seed(seed=j+1)
    unif_random_gauss = np.random.random_sample(size=rnd_quantity)

    x_gauss = gaussian_inv_cdf(unif_random_gauss)
    
    ### Third distribution random numbers ###
    np.random.seed(seed=j+2)
    unif_random_third = np.random.random_sample(size=rnd_quantity)
    
    x_third = scipy_exerc1_distrib3_inv_cdf(unif_random_third)
    
    ###########################################
    ############# COMPUTING MEANS #############
    ###########################################
    for fn in all_func:
        means_unif.append(fn(x_unif))
        means_gauss.append(fn(x_gauss))
        means_third.append(fn(x_third))
        
    means_unif_all.append(means_unif)
    means_gauss_all.append(means_gauss)
    means_third_all.append(means_third)
    
    j+=1

Now, let us compute the estimators expected values by taking the mean of all the computed values in each sample by each estimator.

In [18]:
est_expected_values_unif = []
est_expected_values_gauss = []
est_expected_values_third = []

j=0
for k in range(0, len(means_unif_all[0])):
    expec = sum([item[j] for item in means_unif_all])/samples_quantity
    est_expected_values_unif.append(expec)    
    j+=1

j=0
for k in range(0, len(means_gauss_all[0])):
    expec = sum([item[j] for item in means_gauss_all])/samples_quantity
    est_expected_values_gauss.append(expec)    
    j+=1
    
j=0
for k in range(0, len(means_third_all[0])):
    expec = sum([item[j] for item in means_third_all])/samples_quantity
    est_expected_values_third.append(expec)    
    j+=1

Now, let us check Bias.

In [19]:
df_bias = pd.DataFrame()

df_bias['Estimator'] = pd.Series(data=all_func_names)
df_bias['Unif. Est.Expec.-Expected'] = pd.Series(data=((x - 0.5) for x in est_expected_values_unif))
df_bias['Gauss. Est.Expec.-Expected'] = pd.Series(data=((x - mu_gauss) for x in est_expected_values_gauss))
df_bias['Third. Est.Expec.-Expected'] = pd.Series(data=((x - expected_third) for x in est_expected_values_third))

df_bias

,Estimator,Unif. Est.Expec.-Expected,Gauss. Est.Expec.-Expected,Third. Est.Expec.-Expected
0,est_1,-0.000096,-0.000242,-0.001598
1,est_2,0.000070,0.000276,0.001160
2,est_3,0.017142,-0.000251,0.359612
3,est_4,1.300000,1.800000,-8.676692
4,est_5,-0.126109,NaN,-1.163942
5,est_6,-0.467722,-2.042483,-8.133050
6,est_7,-0.000046,-0.000755,0.846939
7,est_8,-0.000164,-0.000608,-0.003579


So, as it is possible to see, estimators 1, 2 and 8 seem to be unbiased for all the distributions.

## Checking Efficiency

An estimator is efficient if its variance is small.

The variance of an estimator can be defined as $var(\hat{\theta}) = E((\hat{\theta} - E(\hat{\theta}))^2)$. I will compute it using the same technique I used before, that is, computing the expectation value of this quantity as the mean of the computations over each sample.

In [20]:
est_var_unif = []
est_var_gauss = []
est_var_third = []

j=0
for k in range(0, len(means_unif_all[0])):
    expec_val = est_expected_values_unif[j]           #Defining the expected value of the jth estimator.
    new_list = [item[j] for item in means_unif_all]   #Getting the jth member of all samples.
    new_list = [x - expec_val for x in new_list]      #Subtracting the jth member from the expec_val.
    new_list = [x**2 for x in new_list]               #Squaring the previous result.

    expec = sum(new_list)/samples_quantity            #Finally computing the variance.  
    
    est_var_unif.append(expec)
    j+=1
    
j=0
for k in range(0, len(means_gauss_all[0])):
    expec_val = est_expected_values_gauss[j]           #Defining the expected value of the jth estimator.
    new_list = [item[j] for item in means_gauss_all]   #Getting the jth member of all samples.
    new_list = [x - expec_val for x in new_list]      #Subtracting the jth member from the expec_val.
    new_list = [x**2 for x in new_list]               #Squaring the previous result.

    expec = sum(new_list)/samples_quantity            #Finally computing the variance.  
    
    est_var_gauss.append(expec)
    j+=1
    
j=0
for k in range(0, len(means_third_all[0])):
    expec_val = est_expected_values_third[j]           #Defining the expected value of the jth estimator.
    new_list = [item[j] for item in means_third_all]   #Getting the jth member of all samples.
    new_list = [x - expec_val for x in new_list]      #Subtracting the jth member from the expec_val.
    new_list = [x**2 for x in new_list]               #Squaring the previous result.

    expec = sum(new_list)/samples_quantity            #Finally computing the variance.  
    
    est_var_third.append(expec)
    j+=1

Now, let us check efficiency.

In [21]:
df_eff = pd.DataFrame()

df_eff['Estimator'] = pd.Series(data=all_func_names)
df_eff['Unif. Var'] = pd.Series(data=est_var_unif)
df_eff['Gauss. Var'] = pd.Series(data=est_var_gauss)
df_eff['Third Var'] = pd.Series(data=est_var_third)

df_eff

,Estimator,Unif. Var,Gauss. Var,Third Var
0,est_1,2.795991e-03,3.357617e-02,7.018444e-01
1,est_2,8.330143e-03,9.991161e-02,2.088944e+00
2,est_3,2.992143e-03,3.593169e-02,7.510820e-01
3,est_4,2.784444e-24,2.784444e-24,2.784444e-24
4,est_5,4.453444e-03,NaN,8.667171e-01
5,est_6,9.842748e-04,2.453593e-01,1.399991e+00
6,est_7,5.075298e-04,1.255659e-01,1.901694e+00
7,est_8,5.538820e-03,6.640639e-02,1.387655e+00


The most efficient estimators are written below, in ascending order.

Uniform:  2, 8, 5, 3, 1, 6, 7, 4.

Gaussian:    6, 7, 2, 8, 3, 1, 4.   (5 is not defined)

Third:    2, 7, 6, 8, 5, 3, 1, 4.